In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
driver = webdriver.Chrome()
driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
time.sleep(3)
CAFE_NAME = 'rocksoccer'
BOARD_NAME = 'ADs2'
QUERY = '쿠팡'
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)
inp_num = input('저장할 게시물 번호 + 엔터: ')
num = int(inp_num)


In [4]:

url = 'http://m.cafe.daum.net/%s/%s/%s' % (CAFE_NAME, BOARD_NAME, inp_num)
driver.get(url)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
subject = soup.body.find('h3', class_='tit_subject')
import pandas as pd
from bs4 import BeautifulSoup

# 본문과 댓글을 위한 별도의 DataFrame 초기화
df_columns_posts = ['num', 'subject', 'nick', 'write_time', 'views', 'url', 'contents']
df_posts = pd.DataFrame(columns=df_columns_posts)

df_columns_comments = ['cmt_num', 'cmt_writer', 'cmt_time', 'cmt_txt']
df_comments = pd.DataFrame(columns=df_columns_comments)

# csv파일 읽기
urls_df = pd.read_csv('ROCKSOCCER_쿠팡_32pages_inner_links.csv')#수정필요@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
urls = urls_df['url'].tolist()

for url in urls:
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    inp_num = url.split('/')[-1].split('?')[0]
    
    subject = soup.body.find('h3', class_='tit_subject')
    # 게시물 정보 추가 (게시물 본문 크롤링 부분에 추가)
    if subject is None:
        print(url, '지워진 게시물입니다.')
        continue
    else:
        subject = subject.get_text(strip=True)
        if soup.body.find('span', class_='txt_subject').find('span', class_='sr_only').get_text() == '작성자':
            nick = soup.body.find('span', class_='sr_only').next_sibling
        else:
            nick = ''
        num_subject = soup.body.find_all('span', class_='num_subject')
        write_time = num_subject[0].get_text()
        views = num_subject[1].get_text()
        contents = soup.body.find('div', id='article').get_text('\n', strip=True)
        
        # 게시물 정보를 df_posts에 추가
        new_row_post = {'num': inp_num, 'subject': subject, 'nick': nick, 'write_time': write_time, 'views': views, 'url': url, 'contents': contents}
        df_posts = pd.concat([df_posts, pd.DataFrame([new_row_post])], ignore_index=True)
    
    
        # 댓글
        cmt_total = soup.body.find('article', id='mArticle').find('span', class_="num_total").get_text()
        driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments'.format(CAFE_NAME, BOARD_NAME, inp_num))
        time.sleep(1)
        # 페이지 숫자를 확인하기 위해 캡쳐
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 페이지 숫자 확인
        cmt_page_max = int(soup.body.find('span', id="pagingNav").find_all('span', class_="num_page").pop().get_text())
        # 댓글 정보 추가 (댓글 크롤링 부분에 추가)
        for i in range(cmt_page_max):
            j = cmt_page_max-i
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            cmt_all = soup.body.find('ul', id="commentList").find_all('li')
            for k in cmt_all:
                # 기본값을 설정합니다
                cmt_reply = False
                reply_prefix = ''
                
                if k.div.span.find_all('span', class_="txt_bar"):
                    if k['class'] == ['reply_on']:
                        cmt_reply = True
                        reply_prefix = 'ㄴ'
                
                # 삭제된 댓글 처리
                if k.find('strong', class_='tit_info'):
                    if k.find('strong', class_='tit_info').find('span', class_='txt_detail').get_text(strip=True) == '삭제된 댓글입니다.':
                        continue
                
                cmt_num = inp_num + '_' + k['id'].split('comment_')[1]
                cmt_writer_elem = k.div.find('span', class_='username')
                if cmt_writer_elem:
                    cmt_writer = cmt_writer_elem.get_text(strip=True)
                else:
                    continue  # 삭제된 댓글이므로 건너뜁니다.
                    
                cmt_time = k.div.span.find('span', class_="created_at").get_text()
                cmt_txt = k.div.find('span', class_='txt_detail').get_text(strip=True)
                
                # 댓글 정보를 df_comments에 추가
                new_row_comment = {'cmt_num': reply_prefix + cmt_num, 'cmt_writer': cmt_writer, 'cmt_time': cmt_time, 'cmt_txt': cmt_txt}
                df_comments = pd.concat([df_comments, pd.DataFrame([new_row_comment])], ignore_index=True)
            
            if j > 1:
                driver.get('http://m.cafe.daum.net/{0}/{1}/{2}/comments?prev_page={3}&mode=regular&cdepth={4}&page={5}'.format(CAFE_NAME, BOARD_NAME, inp_num, j, '0002100000', (j-1)))
                time.sleep(2)
df_posts.to_csv('posts.csv', index=False)
df_comments.to_csv('comments.csv', index=False)

driver.close()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [96]:
df_posts

,num,subject,nick,write_time,views,url,contents
0,278531,썰호정에서 서기자님 쿠플 관련 재밌는 화두를 던지셨네요.,우주,22.01.13,"2,386",http://m.cafe.daum.net/rocksoccer/ADs2/278531,오늘 쿠팡플레이에서 손흥민의 토트넘 초청해서 K리그와의 경기에 대한 얘기가 나왔었죠...


In [98]:
df_comments

,cmt_num,cmt_writer,cmt_time,cmt_txt
0,278531_21,이수빈,22.01.13,얼마든 전 돈내고 보겠지만 유료화되면 진짜 우리만의 리그 될거 같아서 반대해요. 저...
1,278531_22,상상더하기,22.01.13,올스타전 예매 언제인가요?
2,278531_23,우주작성자 본인 여부작성자,22.01.13,아직 올스타전 한다 안 한다도 확실히 정해진 건 아니에요
3,278531_26,상상더하기,22.01.13,우주아 쿠팡이 리그에 유입되면 대환영이라는 드립입니다
4,278531_27,우주작성자 본인 여부작성자,22.01.13,상상더하기
5,278531_1,서울지동원,22.01.13,저렇게만 되면 좋겠지만 가능성이 있나요?
6,278531_9,우주작성자 본인 여부작성자,22.01.13,"뭐... 그냥 아이디어 수준인 거 같아요.다만, 저번에 유벤투스 때 K리그 위상에도..."
7,278531_2,문근영,22.01.13,쿠팡플레이가 해줄리가 없으니까 기대도 찬성도 못하겠음. 그리고 축구라는 종목에 관심...
8,278531_3,Be FEARLESS,22.01.13,한번 프로연맹이 협상테이블에 앉아서 협상은 해봤으면 싶음쿠플이 어떤 생각인지 궁금하긴 함
9,278531_5,인텔,22.01.13,일단 시도는 해봤으면 좋겠어요.물론 쿠팡입장에서는 해외축구도 딱 몇몇 한국선수 소속...
